In [1]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/audio-iium.zip
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/collections/shuffled-iium.json
# !unzip audio-iium.zip -d iium

In [2]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/audio-wattpad.zip
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/collections/transcript-wattpad.json
# !unzip audio-wattpad.zip -d wattpad

In [3]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/text-audiobook.tar.gz
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/testset-audiobook.tar.gz
# !tar -zxf text-audiobook.tar.gz
# !tar -xf testset-audiobook.tar.gz

In [4]:
import os
from glob import glob
from tqdm import tqdm
import json

In [5]:
import unicodedata
import re

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "'", "-", "z", "0", "1", "x", "2", "q", "*", "5", "3", "4", "6", "9", "8", "7", "%", "$", "\"", "/", "&", ":", "+"]
def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c for c in string if c in vocabs])
    return re.sub(r'[ ]+', ' ', string).strip()

In [6]:
wattpad = []
wavs = glob('wattpad/audio-wattpad/*wav')

with open('transcript-wattpad.json') as fopen:
    transcript = json.load(fopen)
    
for i in tqdm(wavs):
    index = i.split('/')[-1].replace('.wav','')
    text = transcript[int(index)]
    wattpad.append((i, text))

100%|██████████| 146/146 [00:00<00:00, 351128.66it/s]


In [7]:
iium = []
wavs = glob('iium/audio-iium/*wav')

with open('shuffled-iium.json') as fopen:
    transcript = json.load(fopen)
    
for i in tqdm(wavs):
    index = i.split('/')[-1].replace('.wav','')
    text = transcript[int(index)]
    iium.append((i, text))

100%|██████████| 97/97 [00:00<00:00, 258315.87it/s]


In [8]:
audiobook = []
wavs = glob('test-set/*wav')
for i in tqdm(wavs):
    t = '/'.join(i.split('<>')[1:])
    t = t.split('.wav')[0]
    t = t.replace('output-wav', 'output-text')
    with open(f'text-audiobook/{t}.wav.txt') as fopen:
        text = fopen.read()
    audiobook.append((i, text))

100%|██████████| 300/300 [00:00<00:00, 36921.69it/s]


In [9]:
with open('singlish-mixed.json') as fopen:
    data = json.load(fopen)

In [10]:
audios = wattpad + iium + audiobook + data['test']
audios, texts = zip(*audios)

In [11]:
processed_text = [preprocessing_text(t) for t in texts]

In [14]:
with open('mixed-asr-test.json', 'w') as fopen:
    json.dump({'X': audios, 'Y':processed_text}, fopen)

In [ ]:
# import malaya_speech

In [ ]:
# tokenizer = malaya_speech.subword.load('transducer.subword')
# tokenizer

In [ ]:
# malaya_speech.subword.decode(tokenizer, [0, 2, 133, 875])

In [ ]:
# from pydub import AudioSegment
# import numpy as np

# sr = 16000

# def mp3_to_wav(file, sr = sr):
#     audio = AudioSegment.from_file(file)
#     audio = audio.set_frame_rate(sr).set_channels(1)
#     sample = np.array(audio.get_array_of_samples())
#     return malaya_speech.astype.int_to_float(sample), sr

# def generator(maxlen = 18, min_length_text = 2):
#     for i in tqdm(range(len(audios))):
#         try:
#             if audios[i].endswith('.mp3'):
#                 wav_data, _ = mp3_to_wav(audios[i])
#             else:
#                 wav_data, _ = malaya_speech.load(audios[i])
                
#             if (len(wav_data) / sr) > maxlen:
#                 print(f'skipped audio too long {audios[i]}')
#                 continue
                
#             if len(processed_text[i]) < min_length_text:
#                 print(f'skipped text too short {audios[i]}')
#                 continue    

#             yield {
#                 'waveforms': wav_data.tolist(),
#                 'waveform_lens': [len(wav_data)],
#                 'targets': malaya_speech.subword.encode(tokenizer, processed_text[i], add_blank = False),
#             }
#         except Exception as e:
#             print(e)
            
# generator = generator()

In [ ]:
# import os
# import tensorflow as tf

# os.system('rm bahasa-asr-test/data/*')
# DATA_DIR = os.path.expanduser('bahasa-asr-test/data')
# tf.gfile.MakeDirs(DATA_DIR)

In [ ]:
# shards = [{'split': 'dev', 'shards': 10}]

In [ ]:
# import malaya_speech.train as train

# train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'bahasa-asr')